In [0]:
import pandas as pd

# Charger le fichier Excel dans un DataFrame Pandas
pdf = pd.read_excel("/Volumes/workspace/default/data/donnees_bls/2015/allhlcn153.xlsx")

# Convertir en DataFrame Spark pour le Big Data
df_spark = spark.createDataFrame(pdf)
# df_spark.show()


In [0]:
df.display()


In [0]:
pdf.info()

In [0]:
import pandas as pd
import os

# 1. Chemin vers votre dossier principal
root_path = "/Volumes/workspace/default/data/donnees_bls/"

# 2. Boucle simple sur les dossiers d'années
for year_folder in sorted(os.listdir(root_path)):
    year_path = os.path.join(root_path, year_folder)
    
    if os.path.isdir(year_path):
        # Boucle sur les fichiers Excel dans chaque dossier
        for file in os.listdir(year_path):
            if file.endswith(".xlsx"):
                file_path = os.path.join(year_path, file)
                
                # Lecture du fichier avec Pandas
                df = pd.read_excel(file_path)
                
                # Calcul des nulls par colonne
                null_report = df.isnull().sum()
                count = df.shape
                # On n'affiche que les colonnes qui ont au moins 1 null
                null_columns = null_report[null_report > 0]
                
                print(f"--- Fichier: {file} ({year_folder}) ---")
                if not null_columns.empty:
                    print(null_columns)
                    print(count)
                else:
                    print("Aucun NULL trouvé.")
                print("-" * 30)

In [0]:
import pandas as pd
import os

# 1. Configuration des colonnes
# On liste toutes les colonnes "fixes" que vous voulez garder
id_vars = [
    'Area Code', 'St', 'Cnty', 'Own', 'NAICS', 'Year', 'Qtr', 
    'Area Type', 'St Name', 'Area', 'Ownership', 'Industry', 
    'Status Code', 'Establishment Count', 'Total Quarterly Wages', 
    'Average Weekly Wage', 'Employment Location Quotient Relative to U.S.', 
    'Total Wage Location Quotient Relative to U.S.'
]

# Liste complète des noms de mois possibles dans vos fichiers Excel
month_cols_all = [
    'January Employment', 'February Employment', 'March Employment',
    'April Employment', 'May Employment', 'June Employment',
    'July Employment', 'August Employment', 'September Employment',
    'October Employment', 'November Employment', 'December Employment'
]

# 2. Lecture du fichier (exemple sur un fichier de 2015)
folder_path = "/Volumes/workspace/default/data/donnees_bls/2015/"
files = [f for f in os.listdir(folder_path) if f.endswith('.xlsx')]
test_file = os.path.join(folder_path, files[0])

pdf = pd.read_excel(test_file)

# 3. Identifier dynamiquement les colonnes présentes dans ce fichier spécifique
present_id_vars = [c for c in id_vars if c in pdf.columns]
present_month_vars = [c for c in month_cols_all if c in pdf.columns]

# 4. Le MELT
# On transforme les colonnes de mois en deux colonnes : "Mois" et "Valeur Emploi"
df_melted = pdf.melt(
    id_vars=present_id_vars,
    value_vars=present_month_vars,
    var_name="Mois",
    value_name="Valeur Emploi"
)

# 5. Résultat
print(f"Analyse terminée pour : {files[0]}")
display(df_melted.tail())
df_melted.shape

df_melted.columns

In [0]:
df_melted.value_counts("Mois")

In [0]:
import pandas as pd
import os

# 1. Configuration
root_path = "/Volumes/workspace/default/data/donnees_bls/"
all_frames = [] # Liste pour stocker les DataFrames de chaque fichier

id_vars = [
    'Area Code', 'St', 'Cnty', 'Own', 'NAICS', 'Year', 'Qtr', 
    'Area Type', 'St Name', 'Area', 'Ownership', 'Industry', 
    'Status Code', 'Establishment Count', 'Total Quarterly Wages', 
    'Average Weekly Wage', 'Employment Location Quotient Relative to U.S.', 
    'Total Wage Location Quotient Relative to U.S.'
]

month_cols_all = [
    'January Employment', 'February Employment', 'March Employment',
    'April Employment', 'May Employment', 'June Employment',
    'July Employment', 'August Employment', 'September Employment',
    'October Employment', 'November Employment', 'December Employment'
]

# 2. Boucle sur chaque dossier d'année (2015, 2016, ..., 2025)
for year_folder in sorted(os.listdir(root_path)):
    year_path = os.path.join(root_path, year_folder)
    
    # On vérifie que c'est bien un dossier
    if os.path.isdir(year_path):
        print(f"Traitement de l'année : {year_folder}...")
        
        # 3. Boucle sur chaque fichier Excel dans le dossier de l'année
        for file_name in os.listdir(year_path):
            if file_name.endswith('.xlsx'):
                file_path = os.path.join(year_path, file_name)
                
                try:
                    # Lecture du fichier
                    pdf = pd.read_excel(file_path)
                    
                    # Identification des colonnes présentes
                    present_id_vars = [c for c in id_vars if c in pdf.columns]
                    present_month_vars = [c for c in month_cols_all if c in pdf.columns]
                    
                    # Application du Melt
                    df_melted = pdf.melt(
                        id_vars=present_id_vars,
                        value_vars=present_month_vars,
                        var_name="Mois",
                        value_name="Valeur Emploi"
                    )
                    
                    # Ajout à notre liste globale
                    all_frames.append(df_melted)
                    
                except Exception as e:
                    print(f"Erreur sur le fichier {file_name} : {e}")

# 4. Concaténation de TOUS les fichiers en un seul DataFrame
if all_frames:
    df_final = pd.concat(all_frames, ignore_index=True)
    
    print("--- CONSOLIDATION TERMINÉE ---")
    print(f"Nombre total de lignes : {df_final.shape[0]}")
    
    # 5. Conversion en Spark pour profiter de la puissance de Databricks
    spark_df = spark.createDataFrame(df_final)
    display(spark_df)
else:
    print("Aucune donnée n'a été trouvée.")

In [0]:
df_final.columns

In [0]:
df_final.value_counts("Year")

In [0]:
df_final.duplicated().sum()

In [0]:
df_final.isnull().sum()